In [1]:
import pathlib
import os
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

from sklearn.metrics import classification_report

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
df_pca_train = pd.read_csv("../pca_reducted_kvasir_seg/train_pca_kvasir.csv", index_col=0)
df_pca_train

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
cju43jcqim2cp08172dvjvyui,-2829.419859,-4360.309411,-575.059686,1007.094318,4562.158640,-883.438753,-69.868939,1725.233639,-1711.060824,-999.529970,...,-44.676451,114.288722,20.866908,-37.097454,-209.677129,-35.224785,-16.851047,114.504894,72.554146,-41.800830
cju88oh0po9gq0801nge4tgr1,-516.297794,-5386.019764,-1471.249504,2471.775313,-828.526101,2759.242421,256.652930,47.824027,-497.403221,1113.931868,...,4.853969,-16.652698,53.269610,-83.803114,-34.860047,-143.235185,-70.299848,16.283228,90.708129,-1.834795
cju45rj7ln8980850a7821fov,-2989.770061,-2558.644601,-2974.837212,-3068.348387,-3022.624719,-1574.522367,-1070.869399,-45.941306,-100.983416,905.280411,...,-34.311425,94.718882,-20.698240,-203.332440,-53.857884,7.651053,-120.300438,18.094047,84.607145,-263.666354
cju5hwonqedw10801vsd3w6kk,1539.473072,1843.359953,-1086.343915,3024.463496,-2475.832786,1763.051573,-385.294952,-498.515954,-136.304413,2003.789081,...,-17.013456,-192.648105,3.499378,126.607786,-12.634792,-80.285525,-100.132258,-56.747742,54.610658,111.832531
cju8dm2cau2km0818jsv9eeq2,986.914127,-4013.177708,-5294.217474,-1189.628866,1745.764914,2770.921274,-4236.044639,-284.615932,-3233.357083,1928.534849,...,-0.058568,-42.012987,-41.720214,57.370761,-51.408808,17.326587,41.413629,-48.637554,105.648122,16.147830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cju5v8pgplg6k0755rvi2t63h,-336.593048,6217.733553,5752.746816,-749.010361,2215.462002,335.772625,-1283.862929,-1873.475729,966.929337,1184.897512,...,3.689052,289.687525,-35.172332,118.614881,-162.672673,-119.394928,131.552617,-15.317916,-185.079505,-69.209368
cju2htabevq9108015qjei0x7,-5957.484171,-2257.143676,1168.685153,1520.690698,-2616.558776,-4513.929859,1895.456089,-1684.475886,-2405.612723,-1659.792496,...,43.580586,-172.380468,-27.530237,26.518656,20.600893,-114.718442,-20.069842,-100.430861,158.355433,-2.692102
cju5vbo6jldrt0871jf6f1700,-7042.096402,-2534.083826,1257.557689,-6674.670390,-1375.725261,4696.164410,-1451.657865,4032.451929,-73.317809,-1628.076774,...,75.027284,6.894385,7.000588,-151.521375,-59.280503,-129.239142,-200.741809,-51.916895,-110.983819,33.256159
cju414lf2l1lt0801rl3hjllj,3033.751099,5149.715549,-132.858255,-2866.178580,-2935.555271,-571.261223,1882.331310,535.736781,6.803848,784.868660,...,108.845388,154.005633,11.161193,36.286864,-122.886505,-4.421915,94.475729,39.829909,-79.266912,-47.005902


In [4]:
df_pca_test = pd.read_csv("../pca_reducted_kvasir_seg/test_pca_kvasir.csv", index_col=0)
df_pca_test

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
cju7ejm2l2ncl0801wq6y84nw,-3247.901488,-2592.028653,-3601.535889,-3286.960363,126.876492,-1184.741814,-125.394669,853.158284,68.141525,1434.811269,...,17.880308,-14.744217,-5.666535,-36.192289,56.992746,-42.751756,-13.593211,-39.855969,93.853048,-135.654097
cju87xn2snfmv0987sc3d9xnq,-3236.812333,6785.064213,-4037.894249,-1117.141357,-4072.582740,1905.797675,-662.744132,5311.157740,-2927.568563,1923.996461,...,-100.664348,-89.445430,-37.976767,9.643125,-48.799404,34.001873,-115.338072,51.192977,-162.180478,37.480926
cju85rkbnlo1c08503uxcpax1,-678.405249,-4614.874903,-3238.382072,-1482.517852,2003.784046,3762.199413,-1062.834462,-205.499350,45.110220,-140.490884,...,100.164697,-17.001187,-23.912073,-53.638350,-2.047129,-55.388221,-60.287693,-64.868606,35.952600,54.171303
cju84kplnl1y30755ropua1b0,3014.818076,4422.615417,-872.872541,2983.651159,-1798.877180,-1853.045905,-1981.500437,1277.124059,2622.820540,-1973.526671,...,-45.367175,5.478910,79.876102,-28.264808,-20.716685,109.876319,-75.723812,8.274081,58.614832,-79.026188
cju5uhrdwkmsu0817ervv91l8,-5096.548194,-2221.115728,-2332.650132,-4433.462513,2110.834537,-2421.571457,5128.982301,-1185.729704,-1553.497705,1731.164092,...,-76.663647,8.764742,163.155890,18.949317,-43.121119,-98.471144,-5.776646,4.770989,-89.648310,86.410256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cju5enq1tcn1i0755hnkon787,-365.370681,-1284.167975,-2620.629242,3211.540483,4528.056239,112.083685,1951.424697,1449.988889,281.751985,1441.050191,...,-1.769907,-19.567602,-33.388691,68.505512,-7.129114,13.768779,40.527206,38.945671,-20.534976,23.893247
cju5fs6j6d8350801vglraq4u,207.627862,-961.793013,-4227.604562,1349.979667,1834.763674,2329.799920,-827.970417,1677.332458,896.130151,2010.741031,...,-41.706276,-161.904810,1.850170,7.331059,-72.846011,-78.150291,20.064504,-45.318911,-58.376231,19.481426
cju5bf6hxa6m50817rbwettgu,-2524.079319,-1376.768151,-1785.777901,564.444513,3577.435605,1876.974705,-3088.712428,-1631.510404,-2912.904903,1711.603211,...,23.485479,-18.579849,-38.328696,-127.929817,-30.321774,-12.445311,5.539556,-14.064536,-3.147601,12.213137
cju5vcmrqla7i0817x4sp4pqw,-579.226266,1489.744597,510.065782,539.031786,-3757.489594,-4539.790060,3301.900782,2111.154085,3028.076067,-124.431252,...,-65.920193,47.494469,-48.798831,-24.219051,60.389309,120.651327,-110.973316,28.520442,-41.622786,25.881936


In [5]:
# load the images and masks
dir_images_train = pathlib.Path('../../data/kvasir-seg/Kvasir-SEG-processed/train/images')
dir_masks_train = pathlib.Path('../../data/kvasir-seg/Kvasir-SEG-processed/train/masks')
dir_images_test = pathlib.Path('../../data/kvasir-seg/Kvasir-SEG-processed/test/images')
dir_masks_test = pathlib.Path('../../data/kvasir-seg/Kvasir-SEG-processed/test/masks')

In [6]:
train_ids = [f.stem for f in dir_images_train.glob("*.jpg")]
test_ids = [f.stem for f in dir_images_test.glob("*.jpg")]

In [7]:
len(train_ids), len(test_ids)

(762, 182)

In [8]:
class KvasirSegDataset(Dataset):
    def __init__(self, image_dir, mask_dir, pca_data, ids, transform=None):
        """
        Args:
            image_dir (pathlib.Path): Directory with all the images.
            mask_dir (pathlib.Path): Directory with all the masks.
            pca_data (pd.DataFrame): DataFrame containing PCA features.
            ids (list): List of image/mask IDs (without extensions).
            transform (callable, optional): Optional additional transform to be applied on an image/mask.
        """
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.pca_data = pca_data
        self.ids = ids
        self.transform = transform
        self.to_tensor = ToTensor()  # Define ToTensor transformation

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        name = self.ids[idx]
        # Load image and mask
        image = Image.open(self.image_dir / f"{name}.jpg").convert("RGB")
        mask = Image.open(self.mask_dir / f"{name}.jpg").convert("RGB")

        # Apply ToTensor transformation
        image = self.to_tensor(image)
        mask = self.to_tensor(mask)

        # Load PCA features
        image_pca = self.pca_data.loc[name].values.astype(np.float32)

        # Apply additional transformations if provided
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask, torch.tensor(image_pca)


In [9]:
# Create datasets with transformations
dataset_train = KvasirSegDataset(dir_images_train, dir_masks_train, df_pca_train, train_ids)
dataset_test = KvasirSegDataset(dir_images_test, dir_masks_test, df_pca_test, test_ids)

# Create dataloaders
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, num_workers=4)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, num_workers=4)

In [37]:
# create a dataset and dataloader for the data
dataset_train = {}
dataset_test = {}

for name in train_ids:
    image = Image.open(dir_images_train / f"{name}.jpg").convert("RGB")
    mask = Image.open(dir_masks_train / f"{name}.jpg").convert("RGB")
    image_pca = df_pca_train.loc[name].values.astype(np.float32)
    dataset_train[name] = (image, mask, image_pca)

for name in test_ids:
    image = Image.open(dir_images_test / f"{name}.jpg").convert("RGB")
    mask = Image.open(dir_masks_test / f"{name}.jpg").convert("RGB")
    image_pca = df_pca_test.loc[name].values.astype(np.float32)
    dataset_test[name] = (image, mask, image_pca)

In [38]:
dataset_train['cju43jcqim2cp08172dvjvyui']#[0]

(<PIL.Image.Image image mode=RGB size=128x128>,
 <PIL.Image.Image image mode=RGB size=128x128>,
 array([-2.82941992e+03, -4.36030957e+03, -5.75059692e+02,  1.00709430e+03,
         4.56215869e+03, -8.83438782e+02, -6.98689423e+01,  1.72523364e+03,
        -1.71106079e+03, -9.99529968e+02, -1.18987878e+03, -1.01154553e+03,
        -2.80045990e+02, -1.31624847e+02, -8.68060226e+01, -1.13508069e+03,
         1.24385796e+02, -1.20395721e+02,  1.73949326e+02,  4.18080658e+02,
         2.44030334e+02, -2.81244720e+02,  1.51332260e+02,  1.00493851e+02,
        -3.51595947e+02,  2.62214294e+02, -1.84229248e+02,  3.25506317e+02,
         1.25377213e+02,  2.31461792e+02,  5.09045227e+02, -1.21566299e+02,
         7.65681580e+02, -5.04175385e+02, -6.96286804e+02,  7.57882385e+01,
         7.93834457e+01,  5.46454407e+02,  2.65916199e+02, -1.37902161e+02,
        -1.99966064e+02, -7.72396317e+01, -5.40766357e+02, -8.05611511e+02,
        -2.89961517e+02,  1.38726120e+02,  1.29012054e+02, -2.832109

In [17]:
# create a custom neural network model that takes the pca values as input and outputs two cordinates for one point that will be used as a prompt for the sam2 model
# use 3 layers with 100 neurons each and a tanh activation function, the output must be between 0 and 1

class CustomModel(torch.nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = torch.nn.Linear(512, 256)
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc3 = torch.nn.Linear(128, 2)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        # multiply by 128 and convert to int
        x = x * 128
        x = x.int()
        return x
    
model = CustomModel()
model.to(device)

CustomModel(
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [18]:
# test model with a random input
x = torch.randn(512).to(device)
y = model(x)
print(y)

tensor([62, 66], device='cuda:0', dtype=torch.int32)


In [21]:
# test model with the first image pca values from the train dataset
for i, (image, mask, image_pca) in enumerate(train_loader):
    image_pca = image_pca.to(device)
    y = model(image_pca)
    print(y)
    break

tensor([[69, 75],
        [71, 56],
        [66, 78],
        [71, 71],
        [56, 62],
        [53, 67],
        [51, 69],
        [60, 74],
        [64, 73],
        [52, 72],
        [62, 82],
        [58, 71],
        [66, 56],
        [75, 69],
        [58, 58],
        [57, 79]], device='cuda:0', dtype=torch.int32)


In [ ]:
# build the sam2 model
sam2_checkpoint = "../sam2/checkpoints/sam2.1_hiera_tiny.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_t.yaml"
sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=device)
predictor = SAM2ImagePredictor(sam2_model)

In [ ]:
# continue from here: build the pipeline to train the custom model

In [22]:
# Training custom model with REINFORCE

## Hyperparameters
#input_dim = 10
#output_dim = 2  # 2D points
#num_points = 5
#num_epochs = 1000
#batch_size = 32
#learning_rate = 0.001
#
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
## Training Loop
#for epoch in range(num_epochs):
#    # 1. Generate random inputs (images/features)
#    inputs = torch.randn(batch_size, input_dim)  # Simulated inputs
#
#    # 2. model generates points
#    points = model(inputs)  # Shape: (batch_size, num_points, 2)
#
#    # 3. call SAM for segmentation usign the points as prompt (one foreground point for each image)
#    rewards = model(points)  # Shape: (batch_size,)
#
#    # 4. Compute log probabilities of the points (policy log-probabilities)
#    # Assume Gaussian sampling with mean = generated points
#    distribution = torch.distributions.Normal(points, torch.tensor(0.1))
#    log_probs = distribution.log_prob(points).sum(dim=(-1, -2))  # Sum over points and dimensions
#
#    # 5. REINFORCE: Scale log_probs by rewards
#    loss = -torch.mean(rewards * log_probs)  # Negative reward for gradient ascent
#    
#    # 6. Backpropagate and update nn A
#    optimizer.zero_grad()
#    loss.backward()
#    optimizer.step()
#    # Logging
#    if epoch % 100 == 0:
#        print(f"Epoch {epoch}: Loss = {loss.item()}, Mean Reward = {rewards.mean().item()}")


In [23]:
#for i in range(10):
#    image = images[i].squeeze(0)
#    label = labels[i]
#    predictor.set_image(np.array(image.permute(1, 2, 0)))
#    # Integrated Gradients prompt
#    prompt_points = engineered_prompt_coordinates_dict['ig'][i]
#    prompt_labels = [1] * len(prompt_points) # all foreground
#    mask_ig, _, _ = predictor.predict(point_coords=prompt_points, point_labels=prompt_labels, multimask_output=False)
#
#    # Saliency prompt
#    prompt_points = engineered_prompt_coordinates_dict['saliency'][i]
#    prompt_labels = [1] * len(prompt_points) # all foreground
#    mask_saliency, _, _= predictor.predict(point_coords=prompt_points, point_labels=prompt_labels, multimask_output=False)
#
#    # DeepLift prompt
#    prompt_points = engineered_prompt_coordinates_dict['deep_lift'][i]
#    prompt_labels = [1] * len(prompt_points) # all foreground
#    mask_deep_lift, _, _ = predictor.predict(point_coords=prompt_points, point_labels=prompt_labels, multimask_output=False)
#
#    # Auto prompt
#    mask_auto, _, _ = predictor.predict(point_coords=None, point_labels=None, multimask_output=False)
#
#    sam2_segmentations_dict['ig'].append(mask_ig)
#    sam2_segmentations_dict['saliency'].append(mask_saliency)
#    sam2_segmentations_dict['deep_lift'].append(mask_deep_lift)
#    sam2_segmentations_dict['auto'].append(mask_auto)